In [1]:
from StackOverFlowTool import Stack_overflow_tool
tool=Stack_overflow_tool

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import SystemMessage, ToolMessage, HumanMessage, AIMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


class Agent:
    def __init__(self,model,tools,system):
        self.system=system
        graph=StateGraph(AgentState)
        graph.add_node("llm",self.call_groq)
        graph.add_node("StackOverFLow Tool",self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.action_exists,
            {True:"StackOverFLow Tool",False:END}
        )
        graph.add_edge("StackOverFLow Tool","llm")
        graph.set_entry_point("llm")
        self.graph=graph.compile()
        self.tools={t.name:t for t in tools}
        self.model=model.bind_tools(tools)
        
    def action_exists(self,state:AgentState):
        result=state['messages'][-1]
        return len(result.tool_calls)>0
        
        
    def call_groq(self,state:AgentState):
        messages=state["messages"]
        if self.system:
            messages=[SystemMessage(content=self.system)] + messages
        message=self.model.invoke(messages)
        return {'messages':[message]}
    
    def take_action(self,state:AgentState):
        tool_calls=state['messages'][-1].tool_calls
        results=[]
        for t in tool_calls:
            print(f'Calling: {t}')
            result=self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t["id"],name=t["name"],content=str(result)))
        print("Back to the Model !!")
        return {"messages":results}
    

In [3]:
from langchain_groq import ChatGroq
prompt = """
        You are a helpful assistant that answers questions about programming.
        Always use the StackOverFlow tool to answer the question.
        Always feed the tool with the whole user query.
""".strip()

tools=[tool]
model=ChatGroq(model='llama-3.1-8b-instant')
abot=Agent(model,tools,prompt)

In [4]:
print(abot.graph.get_graph().draw_ascii())

                +-----------+             
                | __start__ |             
                +-----------+             
                       *                  
                       *                  
                       *                  
                   +-----+                
                   | llm |.               
                   +-----+ .              
                ***         ...           
               *               .          
             **                 ..        
+--------------------+       +---------+  
| StackOverFLow Tool |       | __end__ |  
+--------------------+       +---------+  


In [5]:
messages=[HumanMessage(content='''I'm getting an error with my Python list comprehension when trying to filter and transform a list of dictionaries. The error says "TypeError: string indices must be integers".''')]
for event in abot.graph.stream({'messages':messages}):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hc73', 'function': {'arguments': '{"query": "Python list comprehension error string indices must be integers when filtering and transforming a list of dictionaries"}', 'name': 'StackOverflow Tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 393, 'total_tokens': 425, 'completion_time': 0.042666667, 'prompt_time': 0.019079251, 'queue_time': 0.04587863500000001, 'total_time': 0.061745918}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9c429c48-c97b-4fd0-a411-1b35128a6698-0', tool_calls=[{'name': 'StackOverflow Tool', 'args': {'query': 'Python list comprehension error string indices must be integers when filtering and transforming a list of dictionaries'}, 'id': 'call_hc73', 'type': 'tool_call'}], usage_metadata={'input_tokens': 393, 'output_tokens': 32, 'total_tokens': 425})]